#                            WebScrapping of YouTube Search results.


In This Markdown we will see how to scrap the YouTube search results of Kishore Kumar, and we will be scrapping the top 10 searching and video details like When the video was uploaded, who is the uploaders, number of up votes, no of down votes, comments.

Step1: Importing the necessary libraries

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import time
import os
import csv
import pandas as pd
from math import ceil
from selenium.webdriver.common.keys import Keys
from time import sleep
import urllib
from pandas.compat import StringIO 
import re
#import urllib.parse
import urllib
import sys

Step2: Defining all the punctuations required to scrap.

In [2]:
def nopunct(my_str):
        punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
        no_punct = ""
        for char in my_str:
            if char not in punctuations:
                no_punct = no_punct + char
        return(no_punct)

Step3: We are providing the path of the chrome driver.

In [4]:
#locationOfWebdriver = "/Users/navyarao/Desktop/chromedriver"
driver = webdriver.Chrome(executable_path=r'C:/Users/Akhilesh/Downloads/chromedriver_win32/chromedriver.exe')


Step4: We are giving the text what to search in the search engine.

In [6]:
textToSearch = 'Telangana Elections 2018'
query = urllib.quote(textToSearch)
#query = urllib.parse.quote(textToSearch)
url = "https://www.facebook.com/search/top/?q=telangana%20elections&epa=SEARCH_BOX" + query
driver.get(url)
driver.execute_script("window.scrollTo(0, 3000)")

Step 5: We are giving the Xpath of the elements to find and we are giving scrapping top 10 results

In [118]:
container=driver.find_elements_by_xpath('//*[(@id = "video-title")]')
contain_top=container[:10]
links=[]
for page in contain_top:
    url=page.get_attribute("href")
    links.append(url)
links

[u'https://www.youtube.com/watch?v=b_iSFNJmAhU',
 u'https://www.youtube.com/watch?v=GSmXU2Q7TU8',
 u'https://www.youtube.com/watch?v=uA67M0Lihz0',
 u'https://www.youtube.com/watch?v=aqBHgUkoZCE',
 u'https://www.youtube.com/watch?v=6vCbtPN9skk',
 u'https://www.youtube.com/watch?v=duee3ROzuKg',
 u'https://www.youtube.com/watch?v=F5KO42OWQV0',
 u'https://www.youtube.com/watch?v=nEnLt3pasxE',
 u'https://www.youtube.com/watch?v=lKVIElw2IZM',
 u'https://www.youtube.com/watch?v=AMuRRXCuy-4']

Step 6: We are defining a loop to select the elements we need to scrap and write to a dataframe.

For Each Video we are extracting the below information:
1) Link of the video 
2) Title of the video
3) Who uploaded the video
4) Total Views on the video
5) When the video was published
6) User of the video
7) Duraion of the video
8) Comments for the video
9) Number of dislikes
10) Number of likes
11) Number of replies for each comment

In [119]:
final = pd.DataFrame()
for url in links:
    driver.get(url)
    time.sleep(10)
    print("=" * 40)  # Shows in terminal when a new airline is being scraped
    print("Scraping " + url)
    try:
        video_title = driver.find_element_by_xpath('//*[(@id = "video-title")]').text
    except:
        video_title=" "
    try:
        uploaded_by=driver.find_element_by_xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "yt-formatted-string", " " ))]').text
    except:
        uploaded_by=" "
    try:
        Views=driver.find_element_by_xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "yt-view-count-renderer", " " ))]').text
    except:
        Views=" "
    try:
        uploaded_Date=driver.find_element_by_xpath('//*[@id="upload-info"]/span').text
    except:
        uploaded_Date=" "
    
    driver.execute_script("window.scrollTo(0, 3000)")
    time.sleep(5)
    elm=driver.find_element_by_xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "has-items", " " ))]')
    elm.click()
    top=driver.find_element_by_xpath('//*[@id="menu"]/a[1]/paper-item/paper-item-body/div[1]')
    top.click()
    
    #driver.execute_script("window.scrollTo(0, 6000)")
    time.sleep(5)
    name=driver.find_elements_by_xpath('//*[(@id = "author-text")]')
    date_of_comment=driver.find_elements_by_xpath('//*[(@id = "published-time-text")]')
    comments=driver.find_elements_by_xpath('//*[(@id = "content-text")]')
    upvotes_cmnts=driver.find_elements_by_xpath('//*[(@id = "vote-count-middle")]')
    dwnvotes_cmnts=driver.find_elements_by_xpath('//*[@id="creator-heart"]')
    reply=driver.find_elements_by_xpath('//*[@id="replies"]')
#User######################################################################################    
    df_names = pd.DataFrame()  
    df1=pd.DataFrame()
    for i in name:
        try:
            txt=i.text
            if(len(txt)>=1):
                df1["User"]=list(pd.read_csv(StringIO(txt)))
            else:
                df1["User"]=list(pd.read_csv(StringIO("null"))) 
        except:
            df1["User"]=list(pd.read_csv(StringIO("null")))    
        df_names=df_names.append(df1)
#Date of Comment######################################################################################    
    df_date = pd.DataFrame()  
    df1=pd.DataFrame()
    for i in date_of_comment:
        try:
            txt=i.text
            if(len(txt)>=1):
                df1["Time"]=list(pd.read_csv(StringIO(txt)))
            else:
                df1["Time"]=list(pd.read_csv(StringIO("null"))) 
        except:
            df1["Time"]=list(pd.read_csv(StringIO("null"))) 
        df_date=df_date.append(df1)
#Comment######################################################################################    
    df_comments = pd.DataFrame()
    df1=pd.DataFrame()    
    for j in comments:
        try:
            cmnt=j.text
            if(len(cmnt)>=1):
                df1["Comment"]=list(pd.read_csv(StringIO(nopunct(cmnt))))
            else:
                df1["Comment"]=list(pd.read_csv(StringIO("null"))) 
        except:
            df1["Comment"]=list(pd.read_csv(StringIO("null"))) 
        df_comments=df_comments.append(df1)
#up votes######################################################################################    
    df_upvotes = pd.DataFrame()  
    df1=pd.DataFrame()         
    for upvotes in upvotes_cmnts:
        try:
            upvot=upvotes.text
            if(len(upvot)>=1):
                df1["Likes(upvotes)"]=list(pd.read_csv(StringIO(upvot)))
            else:
                df1["Likes(upvotes)"]=list(pd.read_csv(StringIO("null"))) 
        except:
            df1["Likes(upvotes)"]=list(pd.read_csv(StringIO("null")))
        df_upvotes=df_upvotes.append(df1)
#down votes######################################################################################    
    df_downvotes = pd.DataFrame()  
    df1=pd.DataFrame()     
    for downvotes in dwnvotes_cmnts:      
        try:
            dwnvot=downvotes.text
            print(dwnvot)
            if(len(dwnvot)>=1):
                df1["Dislikes(down_votes)"]=list(pd.read_csv(StringIO(dwnvot)))
            else:
                df1["Dislikes(down_votes)"]=list(pd.read_csv(StringIO("null"))) 
        except:
            df1["Dislikes(down_votes)"]=list(pd.read_csv(StringIO("null")))
        df_downvotes=df_downvotes.append(df1)
#Replies######################################################################################    
    df_replies = pd.DataFrame()  
    df1=pd.DataFrame()
    for replies in reply:
        try:
            rpl=replies.text
            if(len(rpl)>=1):
                if(rpl == "View reply"):
                    rpl = '1'
                else:
                    rpl = re.search('View (.*) replies',rpl).group(1)
                df1["Replies"]=list(pd.read_csv(StringIO(rpl)))
            else:
                df1["Replies"]=list(pd.read_csv(StringIO("null"))) 
        except:
            df1["Replies"]=list(pd.read_csv(StringIO("null")))
        df_replies=df_replies.append(df1)

    final_url =pd.concat([df_names.reset_index(drop=True),df_date.reset_index(drop=True), df_comments.reset_index(drop=True),df_upvotes.reset_index(drop=True),df_downvotes.reset_index(drop=True),df_replies.reset_index(drop=True)], axis=1)
    final_url["Link"] = url
    final_url["Title"] = video_title
    final_url["Uploaded_by"] = uploaded_by
    final_url["Total_Views"] = Views
    final_url["Published_on"] = uploaded_Date
    final=final.append(final_url)
    cols = final.columns.tolist()
final = final[cols[-5:]+cols[:-5]]

Scraping https://www.youtube.com/watch?v=b_iSFNJmAhU




















Scraping https://www.youtube.com/watch?v=GSmXU2Q7TU8




















Scraping https://www.youtube.com/watch?v=uA67M0Lihz0




















Scraping https://www.youtube.com/watch?v=aqBHgUkoZCE




















Scraping https://www.youtube.com/watch?v=6vCbtPN9skk




















Scraping https://www.youtube.com/watch?v=duee3ROzuKg




















Scraping https://www.youtube.com/watch?v=F5KO42OWQV0




















Scraping https://www.youtube.com/watch?v=nEnLt3pasxE




















Scraping https://www.youtube.com/watch?v=lKVIElw2IZM




















Scraping https://www.youtube.com/watch?v=AMuRRXCuy-4






















In [120]:
final

,Link,Title,Uploaded_by,Total_Views,Published_on,User,Time,Comment,Likes(upvotes),Dislikes(down_votes),Replies
0,https://www.youtube.com/watch?v=b_iSFNJmAhU,Best Of R D Burman Songs - Old Hindi Bollywood...,Rajshri,"17,688,867 views","Published on Feb 25, 2013",Hitler Rage,3 years ago,SADLY KISHORE KUMAR died at the age of 58yrs ...,143,null,23
1,https://www.youtube.com/watch?v=b_iSFNJmAhU,Best Of R D Burman Songs - Old Hindi Bollywood...,Rajshri,"17,688,867 views","Published on Feb 25, 2013",Saikat Kundu,5 months ago,Kishore Kumar is a genius The people who disli...,58,null,6
2,https://www.youtube.com/watch?v=b_iSFNJmAhU,Best Of R D Burman Songs - Old Hindi Bollywood...,Rajshri,"17,688,867 views","Published on Feb 25, 2013",Sudhanshu,1 year ago,Serenity Guys play this playlist in the mid ni...,25,null,1
3,https://www.youtube.com/watch?v=b_iSFNJmAhU,Best Of R D Burman Songs - Old Hindi Bollywood...,Rajshri,"17,688,867 views","Published on Feb 25, 2013",SK Aryan,1 year ago,Kishore da lives in our hearts will remain for...,54,null,9
4,https://www.youtube.com/watch?v=b_iSFNJmAhU,Best Of R D Burman Songs - Old Hindi Bollywood...,Rajshri,"17,688,867 views","Published on Feb 25, 2013",Sazani Thapa,1 year ago,These songs give me chillllss 🔥 lit songs Tho...,18,null,1
5,https://www.youtube.com/watch?v=b_iSFNJmAhU,Best Of R D Burman Songs - Old Hindi Bollywood...,Rajshri,"17,688,867 views","Published on Feb 25, 2013",Mazhar Ali Bhatt,2 years ago,Kishore da Incredible man Singer+ Comedian + ...,192,null,12
6,https://www.youtube.com/watch?v=b_iSFNJmAhU,Best Of R D Burman Songs - Old Hindi Bollywood...,Rajshri,"17,688,867 views","Published on Feb 25, 2013",Ankan Dutta,2 years ago,Happy Birthday The Legendary Kishor Kumar,18,null,2
7,https://www.youtube.com/watch?v=b_iSFNJmAhU,Best Of R D Burman Songs - Old Hindi Bollywood...,Rajshri,"17,688,867 views","Published on Feb 25, 2013",Jaswinder Singh,2 years ago,Only a dumb person with 0 knowledge of music c...,94,null,13
8,https://www.youtube.com/watch?v=b_iSFNJmAhU,Best Of R D Burman Songs - Old Hindi Bollywood...,Rajshri,"17,688,867 views","Published on Feb 25, 2013",Arvind Shrivastav,2 years ago,the best songs of the century,196,null,20
9,https://www.youtube.com/watch?v=b_iSFNJmAhU,Best Of R D Burman Songs - Old Hindi Bollywood...,Rajshri,"17,688,867 views","Published on Feb 25, 2013",0208Habib,3 years ago,KISHORE KUMAR THE SINGER WHO MAKES ME CRYING L...,35,null,5


Step 7: We are writing to .CSV file

In [124]:
final.to_csv("C:/Users/tsvvs/Desktop/navya/Youtube Scrapping/final.csv", sep='\t')


In [104]:
J =1
for i in final.Link.unique():
    sub = final[(final.Link == i)]
    print("=" * 120) 
    print("Link",J)
    print("=" * 120) 
    print("Link : ",str(i))
    print("Title : ",str(final.Title.unique()))
    print("Uploaded by : ",str(final.Uploaded_by.unique()))
    print("Total Views : ",str(final.Total_Views.unique()))
    print("Published on : ",str(final.Published_on.unique()))
    print("-" * 120) 
    print("Comments")
    sub = sub[sub.columns.tolist()[5:]]
    print(sub)
    J = J+1

('Link', 1)
('Link : ', 'https://www.youtube.com/watch?v=b_iSFNJmAhU')
('Title : ', "[u'Best Of R D Burman Songs - Old Hindi Bollywood Songs | Audio Jukebox | R D Burman Songs'\n u'Ke Pag Ghungaroo Baandh [HD] | Amitabh Bachchan | Smita Patil | Namak Halal | Bappi Lahiri'\n u'All Anand Songs (HD) - Rajesh Khanna - Amitabh Bachchan - Lata Mangeshkar - Mukesh - Manna Dey'\n u'Ye Shaam Mastani - Kati Patang - Rajesh Khanna & Asha Parekh - Old Hindi Song']")
('Uploaded by : ', "[u'Rajshri' u'Bollywood Classics' u'Saregama Music' u'#kishorekumar'\n u'Indrakr90']")
('Total Views : ', "[u'17,687,612 views' u'21,004,943 views' u'3,822,587 views'\n u'4,206,397 views' u'1,000,569 views']")
('Published on : ', "[u'Published on Feb 25, 2013' u'Published on Nov 26, 2015'\n u'Published on May 4, 2017' u'Published on Oct 1, 2017'\n u'Published on Mar 31, 2012']")
------------------------------------------------------------------------------------------------------------------------
Comments
         